In [1]:
#Ellenőrizzuk mi van a taxis adatbázisban
#Exploratory Data Analysis (Feltáró adatelemzés)
#CTRL nyomva tartva -> GoToDefinition
import pandas as pd
import requests
import os
from dateutil.relativedelta import relativedelta
from datetime import datetime as dt

pd.set_option('display.max_columns',30)

data_out = r'../data/'


In [2]:
api_token = os.environ.get('CHICAGO_API_TOKEN')
headers = {"X-App-Token": api_token}

endDate = dt.now().strftime("%Y-%m-%d")
startDate = (dt.now() - relativedelta(months=2)).strftime("%Y-%m-%d")


#startDate = dt(2023,9,8).strftime("%Y-%m-%d")

#url = f"https://data.cityofchicago.org/resource/wrvz-psew.json?$where=trip_start_timestamp >= '{startDate}T00:00:00' AND trip_start_timestamp <= '{startDate}T23:59:59'&$limit=30000&$$app_token={api_token}"
#url = f"https://data.cityofchicago.org/resource/wrvz-psew.json?$where=trip_start_timestamp >= '{startDate}T00:00:00' AND trip_start_timestamp <= '{startDate}T23:59:59'&$limit=30000"

url_old = f"https://data.cityofchicago.org/resource/wrvz-psew.json?$limit=213000000"
url_new = f"https://data.cityofchicago.org/resource/ajtu-isnz.json?$limit=213000000"

params = f"$where=trip_start_timestamp >= '{startDate}T00:00:00' AND trip_start_timestamp <= '{endDate}T23:59:59'&$limit=30000" # ? nem kell, ha így adod meg
response_old = requests.get(url_old)
response_new = requests.get(url_new)
data_old = response_old.json()
data_new = response_new.json()
response_old, response_new

In [3]:
len(data_new)
data_new_df = pd.DataFrame(data_new)
data_old_df = pd.DataFrame(data_old)


In [4]:
data_new_df.head(5)

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area,dropoff_community_area,fare,tips,tolls,extras,trip_total,payment_type,company,pickup_centroid_latitude,pickup_centroid_longitude,pickup_centroid_location,dropoff_centroid_latitude,dropoff_centroid_longitude,dropoff_centroid_location,pickup_census_tract,dropoff_census_tract
0,f8357229b5314b16f6e35506e285f442c206d56d,d84c3b3515e9b0be353baccf4cfba1e0cef7e656261dc5...,2024-04-01T00:00:00.000,2024-04-01T00:30:00.000,1740,15.3,56,7,38.75,8.55,0,4,51.3,Credit Card,Taxi Affiliation Services,41.79259236,-87.769615453,"{'type': 'Point', 'coordinates': [-87.76961545...",41.922686284,-87.649488729,"{'type': 'Point', 'coordinates': [-87.64948872...",NaN,NaN
1,30f9c5d75d4078691c0aac2e8fce1214d6a61bef,d5c07ff3802aceed5373a10ec72eef292113d324257906...,2024-04-01T00:00:00.000,2024-04-01T00:15:00.000,1200,13,56,8,29.5,6.8,0,4,40.3,Credit Card,Taxi Affiliation Services,41.79259236,-87.769615453,"{'type': 'Point', 'coordinates': [-87.76961545...",41.899602111,-87.633308037,"{'type': 'Point', 'coordinates': [-87.63330803...",NaN,NaN
2,3ac06bf3372ec2d118e0f93d3b43d1ed3380552f,8e08799f421f71f00bb9ded6011b7e5369296773e81ad0...,2024-04-01T00:00:00.000,2024-04-01T00:15:00.000,1243,13.06,76,NaN,33.25,5,0,25,63.75,Credit Card,Taxicab Insurance Agency Llc,41.980264315,-87.913624596,"{'type': 'Point', 'coordinates': [-87.91362459...",NaN,NaN,NaN,NaN,NaN
3,6786690ea138b84b5b6512d3e18914e398aafd0c,8a579afa6df0914d872dac898a493e2d6c36acbe25b73f...,2024-04-01T00:00:00.000,2024-04-01T00:15:00.000,1320,14.9,NaN,NaN,37.5,3,0,4,44.5,Credit Card,"Taxicab Insurance Agency, LLC",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,490229cee7393e29c490c982a0da1a6a53e5d48c,27f4e362035594e6144066180cd9f740961885b33231bd...,2024-04-01T00:00:00.000,2024-04-01T00:15:00.000,1525,17.92,76,32,44.5,9.8,0,4,58.8,Credit Card,Flash Cab,41.97907082,-87.903039661,"{'type': 'Point', 'coordinates': [-87.90303966...",41.884987192,-87.620992913,"{'type': 'Point', 'coordinates': [-87.62099291...",17031980000,17031320100


In [5]:
data_old_df.tail(5)

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,fare,tips,tolls,extras,trip_total,payment_type,company,pickup_centroid_latitude,pickup_centroid_longitude,pickup_centroid_location,dropoff_centroid_latitude,dropoff_centroid_longitude,dropoff_centroid_location,:@computed_region_vrxf_vc4k
99995,1e2202ba1e8e8bebc964ca54b10abe46f199e647,848a1ee7cd6c9d5c24658a35523ec72ae374eef3f562bc...,2023-12-22T18:30:00.000,2023-12-22T18:30:00.000,558,1.13,17031320600,17031320400,32,32,7,0,0,1,8,Cash,Sun Taxi,41.870607372,-87.622172937,"{'type': 'Point', 'coordinates': [-87.62217293...",41.877406123,-87.621971652,"{'type': 'Point', 'coordinates': [-87.62197165...",38
99996,729bd274d2db3b4cd41d4345541f3438e8be5734,ca30b49aab641fe71294a9b03b991b968c7f3e47e11691...,2023-12-22T18:30:00.000,2023-12-22T20:45:00.000,7725,12.38,17031081401,17031081403,8,8,58.75,0,0,0,58.75,Cash,Sun Taxi,41.89503345,-87.619710672,"{'type': 'Point', 'coordinates': [-87.61971067...",41.890922026,-87.618868355,"{'type': 'Point', 'coordinates': [-87.61886835...",37
99997,5e44c2f770adb45c24e7b92d55512c214b0582f0,3f2a24e813695ce63232d6cccfc688ad3875df09b609fa...,2023-12-22T18:30:00.000,2023-12-22T19:00:00.000,1560,7.3,NaN,NaN,8,60,21.25,0,0,0,21.25,Cash,Chicago City Taxi Association,41.899602111,-87.633308037,"{'type': 'Point', 'coordinates': [-87.63330803...",41.836150155,-87.648787952,"{'type': 'Point', 'coordinates': [-87.64878795...",37
99998,08ebda0b2649bbbcacc554c01eb0d59a596316da,c4b8125a4c89c864b1f5dfc240ff17752f46dc27f574cf...,2023-12-22T18:30:00.000,2023-12-22T19:15:00.000,3060,0.6,NaN,NaN,2,76,34.25,0,0,0,34.25,Unknown,Taxi Affiliation Services,42.001571027,-87.695012589,"{'type': 'Point', 'coordinates': [-87.69501258...",41.980264315,-87.913624596,"{'type': 'Point', 'coordinates': [-87.91362459...",20
99999,a77756f7e70ce7ebceaf86a951a8e49c3222ec53,9a3892f81f54fbfcffc54b2c27f94fa864399e135f191e...,2023-12-22T18:30:00.000,2023-12-22T18:45:00.000,411,0.71,17031320100,17031081700,32,8,5.75,2,0,2,10.25,Credit Card,Taxicab Insurance Agency Llc,41.884987192,-87.620992913,"{'type': 'Point', 'coordinates': [-87.62099291...",41.892042136,-87.63186395,"{'type': 'Point', 'coordinates': [-87.63186394...",38


In [ ]:
https://data.cityofchicago.org/resource/ajtu-isnz.json
https://data.cityofchicago.org/resource/wrvz-psew.json

In [15]:
#Ezzel lehet megnézni, hogy mi a tipusa egy vátozónak, vagy egy obj hova tartozik
#type(response)
data_new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 23 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   trip_id                     1000 non-null   object
 1   taxi_id                     1000 non-null   object
 2   trip_start_timestamp        1000 non-null   object
 3   trip_end_timestamp          1000 non-null   object
 4   trip_seconds                1000 non-null   object
 5   trip_miles                  1000 non-null   object
 6   pickup_community_area       965 non-null    object
 7   dropoff_community_area      825 non-null    object
 8   fare                        995 non-null    object
 9   tips                        995 non-null    object
 10  tolls                       995 non-null    object
 11  extras                      995 non-null    object
 12  trip_total                  995 non-null    object
 13  payment_type                1000 non-null   objec

In [3]:
len(data)

0

In [6]:
taxi_trips = pd.concat([data_old_df,data_new_df], ignore_index=True)


In [7]:
taxi_trips.info()
#Itt lehet azokat az oszlopokat kiszűrni, ahol null van.
#Ellenőrizni a duplikációt, mint pl a koordinátáknál

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 24 columns):
 #   Column                       Non-Null Count   Dtype 
---  ------                       --------------   ----- 
 0   trip_id                      200000 non-null  object
 1   taxi_id                      200000 non-null  object
 2   trip_start_timestamp         200000 non-null  object
 3   trip_end_timestamp           199994 non-null  object
 4   trip_seconds                 199960 non-null  object
 5   trip_miles                   199999 non-null  object
 6   pickup_census_tract          69706 non-null   object
 7   dropoff_census_tract         67220 non-null   object
 8   pickup_community_area        195322 non-null  object
 9   dropoff_community_area       181846 non-null  object
 10  fare                         199418 non-null  object
 11  tips                         199418 non-null  object
 12  tolls                        199418 non-null  object
 13  extras        

In [7]:
taxi_trips.describe()
#A mapping táblák létrehozásához segítség. Ha sok kis unique érték van, akkor célszerű mapping táblát csinálni, hogy ne foglaljon annyi helyet.

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area,dropoff_community_area,fare,tips,tolls,extras,trip_total,payment_type,company,pickup_centroid_latitude,pickup_centroid_longitude,pickup_centroid_location,dropoff_centroid_latitude,dropoff_centroid_longitude,dropoff_centroid_location,pickup_census_tract,dropoff_census_tract
count,20799,20799,20799,20799,20793,20799,20239,18863,20749,20749,20749,20749,20749,20799,20799,20252,20252,20252,19088,19088,19088,9928,9644
unique,20799,2439,96,114,3579,2386,77,77,1383,1088,22,186,2497,7,32,149,149,149,175,175,175,72,103
top,244661871466bd53adbd7e28c6fb95e285ac3511,2780ead18beaa862cc67315ddabd9d1acaadcd6da82eba...,2023-09-08T11:45:00.000,2023-09-08T18:00:00.000,0,0,8,8,9,0,0,0,3.25,Credit Card,Taxi Affiliation Services,41.97907082,-87.903039661,"{'type': 'Point', 'coordinates': [-87.90303966...",41.899602111,-87.633308037,"{'type': 'Point', 'coordinates': [-87.63330803...",17031980000,17031839100
freq,1,36,377,391,375,2172,4908,5086,973,10247,20661,12689,450,8341,3952,2175,2175,2175,1530,1530,1530,2175,1147


In [8]:
taxi_trips[taxi_trips['fare'].isna()].sample(5)
#Itt a nem érvényes adatokat lehet ignorálni.

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area,dropoff_community_area,fare,tips,tolls,extras,trip_total,payment_type,company,pickup_centroid_latitude,pickup_centroid_longitude,pickup_centroid_location,dropoff_centroid_latitude,dropoff_centroid_longitude,dropoff_centroid_location,pickup_census_tract,dropoff_census_tract
10984,c6dfda3a09e071969e41b05b463c063a45a6a3d0,286e28223cab05d01037fb52fead0f36d84f4b00e159c0...,2023-09-08T13:45:00.000,2023-09-08T13:45:00.000,562,2.63,8,7,NaN,NaN,NaN,NaN,NaN,Cash,Sun Taxi,41.89503345,-87.619710672,"{'type': 'Point', 'coordinates': [-87.61971067...",41.922082541,-87.634156093,"{'type': 'Point', 'coordinates': [-87.63415609...",17031081401,17031071400
11285,c122eef872c88c77a1e1b840913f9d22b2744e4a,286e28223cab05d01037fb52fead0f36d84f4b00e159c0...,2023-09-08T13:30:00.000,2023-09-08T13:30:00.000,219,0.48,8,8,NaN,NaN,NaN,NaN,NaN,Cash,Sun Taxi,41.892507781,-87.626214906,"{'type': 'Point', 'coordinates': [-87.62621490...",41.89503345,-87.619710672,"{'type': 'Point', 'coordinates': [-87.61971067...",17031081500,17031081401
9792,628d2f7b441f0aae78442570c95ad5648563efaa,875af0d5f0bf4751ef334d77028966ba1d166cabebd0c3...,2023-09-08T14:45:00.000,2023-09-08T14:45:00.000,170,0.34,8,8,NaN,NaN,NaN,NaN,NaN,Cash,Chicago Independents,41.892507781,-87.626214906,"{'type': 'Point', 'coordinates': [-87.62621490...",41.892507781,-87.626214906,"{'type': 'Point', 'coordinates': [-87.62621490...",17031081500,17031081500
17778,a884da9dad61a02a3f911d68ae0835411d6b0201,286e28223cab05d01037fb52fead0f36d84f4b00e159c0...,2023-09-08T08:30:00.000,2023-09-08T08:30:00.000,NaN,0,76,76,NaN,NaN,NaN,NaN,NaN,Cash,Sun Taxi,41.97907082,-87.903039661,"{'type': 'Point', 'coordinates': [-87.90303966...",41.97907082,-87.903039661,"{'type': 'Point', 'coordinates': [-87.90303966...",17031980000,17031980000
11238,ed869257d2a0ef2a18016f29b231cb3fc74b28eb,2407ff99e367ceb469a8cc6663e6ad6ec51fb0f401d370...,2023-09-08T13:30:00.000,2023-09-08T14:15:00.000,2297,25.44,76,24,NaN,NaN,NaN,NaN,NaN,Cash,Taxicab Insurance Agency Llc,41.97907082,-87.903039661,"{'type': 'Point', 'coordinates': [-87.90303966...",41.892658108,-87.652534484,"{'type': 'Point', 'coordinates': [-87.65253448...",17031980000,17031243500


Transformations : deal with NaN values

In [9]:
taxi_trips.drop(['pickup_census_tract', 'dropoff_census_tract'], axis=1, inplace=True)
#mert a fele üres

In [10]:
taxi_trips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20799 entries, 0 to 20798
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   trip_id                     20799 non-null  object
 1   taxi_id                     20799 non-null  object
 2   trip_start_timestamp        20799 non-null  object
 3   trip_end_timestamp          20799 non-null  object
 4   trip_seconds                20793 non-null  object
 5   trip_miles                  20799 non-null  object
 6   pickup_community_area       20239 non-null  object
 7   dropoff_community_area      18863 non-null  object
 8   fare                        20749 non-null  object
 9   tips                        20749 non-null  object
 10  tolls                       20749 non-null  object
 11  extras                      20749 non-null  object
 12  trip_total                  20749 non-null  object
 13  payment_type                20799 non-null  ob

In [11]:
#Koordináták duplikációja miatt
taxi_trips.drop(['pickup_centroid_location', 'dropoff_centroid_location'], axis=1, inplace=True)

In [12]:
taxi_trips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20799 entries, 0 to 20798
Data columns (total 19 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   trip_id                     20799 non-null  object
 1   taxi_id                     20799 non-null  object
 2   trip_start_timestamp        20799 non-null  object
 3   trip_end_timestamp          20799 non-null  object
 4   trip_seconds                20793 non-null  object
 5   trip_miles                  20799 non-null  object
 6   pickup_community_area       20239 non-null  object
 7   dropoff_community_area      18863 non-null  object
 8   fare                        20749 non-null  object
 9   tips                        20749 non-null  object
 10  tolls                       20749 non-null  object
 11  extras                      20749 non-null  object
 12  trip_total                  20749 non-null  object
 13  payment_type                20799 non-null  ob

Transformation : renaming community area colunms

In [13]:
taxi_trips.rename(columns={'pickup_community_area' : 'pickup_community_area_id',
                           'dropoff_community_area' : 'dropoff_community_area_id'
                           },inplace=True)

In [14]:
taxi_trips.columns

Index(['trip_id', 'taxi_id', 'trip_start_timestamp', 'trip_end_timestamp',
       'trip_seconds', 'trip_miles', 'pickup_community_area_id',
       'dropoff_community_area_id', 'fare', 'tips', 'tolls', 'extras',
       'trip_total', 'payment_type', 'company', 'pickup_centroid_latitude',
       'pickup_centroid_longitude', 'dropoff_centroid_latitude',
       'dropoff_centroid_longitude'],
      dtype='object')

Transformation : Create helper col for weather

In [15]:
taxi_trips['trip_start_timestamp'] = pd.to_datetime(taxi_trips['trip_start_timestamp'])
taxi_trips['datetime_for_weather'] = taxi_trips['trip_start_timestamp'].dt.floor('H')

In [16]:
taxi_trips['datetime_for_weather']

0       2023-09-08 23:00:00
1       2023-09-08 23:00:00
2       2023-09-08 23:00:00
3       2023-09-08 23:00:00
4       2023-09-08 23:00:00
                ...        
20794   2023-09-08 00:00:00
20795   2023-09-08 00:00:00
20796   2023-09-08 00:00:00
20797   2023-09-08 00:00:00
20798   2023-09-08 00:00:00
Name: datetime_for_weather, Length: 20799, dtype: datetime64[ns]

Transformations : check joining trips to weather

In [17]:
#Get weather data
url = f"https://archive-api.open-meteo.com/v1/era5"
startDate = '2023-09-08'
params = {
    "latitude" : 41.85,
    "longitude" : -87.65,
    "start_date" : startDate,
    "end_date" : startDate,
    "hourly" : "temperature_2m,wind_speed_10m,precipitation,rain"
}


response = requests.get(url, params=params)
weather_data = response.json()

weather_filtered = {
    'datetime': weather_data['hourly']['time'],
    'temperature' : weather_data['hourly']['temperature_2m'],
    'wind' : weather_data['hourly']['wind_speed_10m'],
    'precipitation' : weather_data['hourly']['precipitation'],
    'rain' : weather_data['hourly']['rain']
}
weather_df = pd.DataFrame(weather_filtered)
weather_df['datetime'] = pd.to_datetime(weather_df['datetime'])




In [18]:
weather_df.head()

,datetime,temperature,wind,precipitation,rain
0,2023-09-08 00:00:00,18.8,14.2,0.0,0.0
1,2023-09-08 01:00:00,18.5,13.9,0.0,0.0
2,2023-09-08 02:00:00,18.5,14.1,0.0,0.0
3,2023-09-08 03:00:00,18.6,13.3,0.0,0.0
4,2023-09-08 04:00:00,18.4,17.3,0.0,0.0


In [19]:
#Merging
taxi_trips_with_weather = taxi_trips.merge(weather_df, left_on='datetime_for_weather', right_on='datetime')
taxi_trips_with_weather.drop(['datetime_for_weather', 'datetime'], axis=1, inplace=True)
taxi_trips_with_weather.head()

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area_id,dropoff_community_area_id,fare,tips,tolls,extras,trip_total,payment_type,company,pickup_centroid_latitude,pickup_centroid_longitude,dropoff_centroid_latitude,dropoff_centroid_longitude,temperature,wind,precipitation,rain
0,244661871466bd53adbd7e28c6fb95e285ac3511,57fa54b44d1e25d82c1de3200158c2475a894490b1c32c...,2023-09-08 23:45:00,2023-09-09T00:00:00.000,965,4.19,24,32,35,7.1,0,0,42.6,Credit Card,Sun Taxi,41.901206994,-87.676355989,41.878865584,-87.625192142,18.0,20.2,0.0,0.0
1,23f5115ef99a29307567bb35b3faaae761543db7,4a263f78f56815be4acd38658af8fc1824ce4e15f7ec81...,2023-09-08 23:45:00,2023-09-09T00:00:00.000,780,2.7,24,23,10.5,0,0,0,10.5,Cash,Taxi Affiliation Services,41.901206994,-87.676355989,41.900069603,-87.720918238,18.0,20.2,0.0,0.0
2,2376339091c00c12857f09362f5a97d7e4cc4a6f,a9263292f54a406cd0fb2bac808b9aaca91f770ad5e0fd...,2023-09-08 23:45:00,2023-09-08T23:45:00.000,240,0.6,32,32,5,0,0,4,9,Cash,Taxi Affiliation Services,41.877406123,-87.621971652,41.870607372,-87.622172937,18.0,20.2,0.0,0.0
3,1f48cd63505fe83e65f15c64a56abf379409dc85,61d83eb1d34ccd8dac5bdd962bc13e2ac077b0c7ecbf1d...,2023-09-08 23:45:00,2023-09-08T23:45:00.000,420,2.9,76,NaN,17,5.38,0,4,26.88,Credit Card,Globe Taxi,41.97907082,-87.903039661,41.982775009,-87.8773054,18.0,20.2,0.0,0.0
4,217da39f98bd50cb5773d994d34182896f583bf8,694d1417f20bc465bf6f926a2893ffda7512bba4d93021...,2023-09-08 23:45:00,2023-09-09T00:15:00.000,1749,11.97,76,NaN,31.5,0,0,5,37,Credit Card,Taxicab Insurance Agency Llc,41.980264315,-87.913624596,NaN,NaN,18.0,20.2,0.0,0.0


Transformation : data type conversions

In [20]:
taxi_trips_with_weather.head(1)

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area_id,dropoff_community_area_id,fare,tips,tolls,extras,trip_total,payment_type,company,pickup_centroid_latitude,pickup_centroid_longitude,dropoff_centroid_latitude,dropoff_centroid_longitude,temperature,wind,precipitation,rain
0,244661871466bd53adbd7e28c6fb95e285ac3511,57fa54b44d1e25d82c1de3200158c2475a894490b1c32c...,2023-09-08 23:45:00,2023-09-09T00:00:00.000,965,4.19,24,32,35,7.1,0,0,42.6,Credit Card,Sun Taxi,41.901206994,-87.676355989,41.878865584,-87.625192142,18.0,20.2,0.0,0.0


In [21]:
#Az AWS nem a df-ből, hanem a csv-ből fog dolgozni. A memória használatot is lehet a megfelelő típus konvertálásokkal csökkenteni.
taxi_trips_with_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20799 entries, 0 to 20798
Data columns (total 23 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   trip_id                     20799 non-null  object        
 1   taxi_id                     20799 non-null  object        
 2   trip_start_timestamp        20799 non-null  datetime64[ns]
 3   trip_end_timestamp          20799 non-null  object        
 4   trip_seconds                20793 non-null  object        
 5   trip_miles                  20799 non-null  object        
 6   pickup_community_area_id    20239 non-null  object        
 7   dropoff_community_area_id   18863 non-null  object        
 8   fare                        20749 non-null  object        
 9   tips                        20749 non-null  object        
 10  tolls                       20749 non-null  object        
 11  extras                      20749 non-null  object    

In [22]:
taxi_trips_with_weather.dropna(inplace=True)

data_types = {
    'trip_end_timestamp' : 'datetime64[ns]',
    'trip_seconds': 'int32',
    'trip_miles': 'float',
    'pickup_community_area_id': 'int8',
    'dropoff_community_area_id': 'int8',
    'fare': 'float',
    'tips': 'float',
    'extras': 'float',
    'trip_total': 'float',

}
taxi_trips_with_weather = taxi_trips_with_weather.astype(data_types)

In [23]:
taxi_trips_with_weather.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18685 entries, 0 to 20798
Data columns (total 23 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   trip_id                     18685 non-null  object        
 1   taxi_id                     18685 non-null  object        
 2   trip_start_timestamp        18685 non-null  datetime64[ns]
 3   trip_end_timestamp          18685 non-null  datetime64[ns]
 4   trip_seconds                18685 non-null  int32         
 5   trip_miles                  18685 non-null  float64       
 6   pickup_community_area_id    18685 non-null  int8          
 7   dropoff_community_area_id   18685 non-null  int8          
 8   fare                        18685 non-null  float64       
 9   tips                        18685 non-null  float64       
 10  tolls                       18685 non-null  object        
 11  extras                      18685 non-null  float64       


Memory usage

In [24]:
taxi_trips.memory_usage(deep=True).sum() / 1024

26759.02734375

In [25]:
taxi_trips_with_weather.memory_usage(deep=True).sum() / 1024

15661.6123046875

Sanity checks

In [26]:
#Megkeresi az értéktartományokat, kiugró értékeket (outliers) 

In [27]:
taxi_trips_with_weather[taxi_trips_with_weather['trip_end_timestamp'] == taxi_trips_with_weather['trip_end_timestamp'].max()]

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area_id,dropoff_community_area_id,fare,tips,tolls,extras,trip_total,payment_type,company,pickup_centroid_latitude,pickup_centroid_longitude,dropoff_centroid_latitude,dropoff_centroid_longitude,temperature,wind,precipitation,rain
830,b63045ac1d6ca24563cffa562d9220c73ca0a471,15ebaceb1f5e522414ecc2fb3af8775a525ccbb8427e69...,2023-09-08 22:30:00,2023-09-09 17:00:00,66895,0.0,34,34,50.0,0.0,0,0.0,50.5,Credit Card,5 Star Taxi,41.842076117,-87.633973422,41.842076117,-87.633973422,18.4,21.3,0.0,0.0


In [28]:
taxi_trips_with_weather.nlargest(10, 'fare')

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area_id,dropoff_community_area_id,fare,tips,tolls,extras,trip_total,payment_type,company,pickup_centroid_latitude,pickup_centroid_longitude,dropoff_centroid_latitude,dropoff_centroid_longitude,temperature,wind,precipitation,rain
19368,7eabbfbdf27dddc9b7ba2c9dda274e6fe5d53cf6,2f1de337941c9b9db18083ebe41474267c955401ea931e...,2023-09-08 06:30:00,2023-09-08 07:00:00,1680,0.00,8,76,754.0,0.00,0,0.0,754.00,Cash,"Taxicab Insurance Agency, LLC",41.890922026,-87.618868355,41.97907082,-87.903039661,17.9,14.4,1.0,1.0
4043,127ae62d3d93ff5959466dd2f1d1e4e394b81551,c60633c5d2053dd1c2076286b88196cf83f4fe0ecc1945...,2023-09-08 19:00:00,2023-09-08 19:00:00,19,0.00,7,7,385.0,96.38,0,0.0,481.88,Credit Card,Chicago Independents,41.922686284,-87.649488729,41.922686284,-87.649488729,18.4,24.4,0.1,0.1
19980,bec226feb1f9db929925686a6ec4a82563097304,8023557dc87c352423d1acd20c4f6e8390d29f01902c23...,2023-09-08 03:00:00,2023-09-08 05:45:00,9794,142.72,76,74,329.5,0.00,0,12.5,342.00,Cash,Star North Taxi Management Llc,41.980264315,-87.913624596,41.694878966,-87.713192497,18.6,13.3,0.0,0.0
12574,af2bf28364c6a2d166c9d11b2f412b42ada5d958,4d97a6adfef73877de02898cc11d1509ccf607b6e12bc3...,2023-09-08 12:30:00,2023-09-08 12:30:00,4,0.00,12,12,235.0,47.30,0,0.0,283.80,Credit Card,Sun Taxi,42.002015464,-87.774994789,42.002015464,-87.774994789,16.8,18.5,0.3,0.3
12669,613b81fc701d0753a8c28563fcedb6f4919d6fee,4d97a6adfef73877de02898cc11d1509ccf607b6e12bc3...,2023-09-08 12:30:00,2023-09-08 12:30:00,6,0.00,12,12,199.0,50.00,0,0.0,250.00,Credit Card,Sun Taxi,42.002015464,-87.774994789,42.002015464,-87.774994789,16.8,18.5,0.3,0.3
12661,6344901bdda6aab54c92eafea118609759b97eb2,4d97a6adfef73877de02898cc11d1509ccf607b6e12bc3...,2023-09-08 12:30:00,2023-09-08 12:30:00,6,0.00,12,12,198.0,39.80,0,0.0,238.80,Credit Card,Sun Taxi,42.002015464,-87.774994789,42.002015464,-87.774994789,16.8,18.5,0.3,0.3
12761,1ec2649d6df6f0c32cc09c7f53063ef30185d7b4,4d97a6adfef73877de02898cc11d1509ccf607b6e12bc3...,2023-09-08 12:30:00,2023-09-08 12:30:00,7,0.00,12,12,195.0,49.00,0,0.0,245.00,Credit Card,Sun Taxi,42.002015464,-87.774994789,42.002015464,-87.774994789,16.8,18.5,0.3,0.3
12986,87d1b3f49fa877ffe231728937031812beee1842,4d97a6adfef73877de02898cc11d1509ccf607b6e12bc3...,2023-09-08 12:15:00,2023-09-08 12:15:00,4,0.00,12,12,195.0,39.20,0,0.0,235.20,Credit Card,Sun Taxi,41.993930128,-87.758353588,41.993930128,-87.758353588,16.8,18.5,0.3,0.3
13130,115039df582f3fd7c57ad9f0071ea44c5bbe88dc,4d97a6adfef73877de02898cc11d1509ccf607b6e12bc3...,2023-09-08 12:15:00,2023-09-08 12:15:00,25,0.00,12,12,188.0,47.25,0,0.0,236.25,Credit Card,Sun Taxi,41.993930128,-87.758353588,41.993930128,-87.758353588,16.8,18.5,0.3,0.3
176,d44c2f724ddd95bcaa0370d830255fb09bd93924,58b1c31975c36cd531a400d3251e6e31d1ede876dce437...,2023-09-08 23:30:00,2023-09-08 23:30:00,58,0.00,76,76,175.0,0.00,0,0.0,175.00,Cash,Blue Ribbon Taxi Association,41.97907082,-87.903039661,41.97907082,-87.903039661,18.0,20.2,0.0,0.0


In [29]:
print(taxi_trips_with_weather.shape)

(18685, 23)


#### Data Modelling

In [30]:
#Mapping table create
taxi_trips_with_weather['payment_type'].unique()

array(['Credit Card', 'Cash', 'Prcard', 'Mobile', 'Unknown', 'No Charge',
       'Dispute'], dtype=object)

In [31]:
payment_type_master = taxi_trips_with_weather['payment_type'].drop_duplicates().reset_index(drop=True)

In [32]:
for i in range(1, len(payment_type_master) + 1):
    print(i)

1
2
3
4
5
6
7


In [33]:
payment_type_master = pd.DataFrame({
    'payment_type_id': range(1, len(payment_type_master) + 1),
    'payment_type':  payment_type_master
})

In [34]:
payment_type_master

,payment_type_id,payment_type
0,1,Credit Card
1,2,Cash
2,3,Prcard
3,4,Mobile
4,5,Unknown
5,6,No Charge
6,7,Dispute


In [35]:
taxi_trips['company'].unique()

array(['Sun Taxi', 'Taxi Affiliation Services', 'Globe Taxi',
       'Taxicab Insurance Agency Llc', 'City Service', 'Flash Cab',
       'Blue Ribbon Taxi Association', '5 Star Taxi', 'Top Cab',
       'Medallion Leasin', 'Choice Taxi Association',
       'Star North Taxi Management Llc', 'Chicago City Taxi Association',
       'Chicago Independents', 'U Taxicab', 'Koam Taxi Association',
       'Patriot Taxi Dba Peace Taxi Associat',
       'Taxicab Insurance Agency, LLC', '312 Medallion Management Corp',
       'Metro Jet Taxi A.', 'Choice Taxi Association Inc', 'Setare Inc',
       'Taxi Affiliation Services Llc - Yell', '4623 - 27290 Jay Kim',
       'Chicago Taxicab', '2733 - 74600 Benny Jona',
       '6574 - Babylon Express Inc.', '3556 - 36214 RC Andrews Cab',
       '4787 - 56058 Reny Cab Co', '5167 - 71969 5167 Taxi Inc',
       'Petani Cab Corp', '4053 - 40193 Adwar H. Nikola'], dtype=object)

In [36]:
company_type_master = taxi_trips_with_weather['company'].drop_duplicates().reset_index(drop=True)
company_type_master = pd.DataFrame({
    'company_id': range(1, len(company_type_master) + 1),
    'company':  company_type_master
})
company_type_master

,company_id,company
0,1,Sun Taxi
1,2,Taxi Affiliation Services
2,3,Flash Cab
3,4,Blue Ribbon Taxi Association
4,5,5 Star Taxi
5,6,Top Cab
6,7,Choice Taxi Association
7,8,City Service
8,9,Star North Taxi Management Llc
9,10,Chicago City Taxi Association


In [37]:
taxi_trips_with_weather[['payment_type', 'company']]

,payment_type,company
0,Credit Card,Sun Taxi
1,Cash,Taxi Affiliation Services
2,Cash,Taxi Affiliation Services
6,Prcard,Flash Cab
7,Credit Card,Blue Ribbon Taxi Association
...,...,...
20794,Mobile,Flash Cab
20795,Cash,Taxi Affiliation Services
20796,Cash,Chicago Independents
20797,Credit Card,Taxi Affiliation Services


In [38]:
taxi_trips_with_weather.memory_usage(deep=True).sum() / 1024

15661.6123046875

In [39]:
taxi_trips_with_weather_id = taxi_trips_with_weather.merge(payment_type_master, on='payment_type')

taxi_trips_with_weather_id = taxi_trips_with_weather_id.merge(company_type_master, on='company')

In [40]:
taxi_trips_with_weather_id.sample(5)

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area_id,dropoff_community_area_id,fare,tips,tolls,extras,trip_total,payment_type,company,pickup_centroid_latitude,pickup_centroid_longitude,dropoff_centroid_latitude,dropoff_centroid_longitude,temperature,wind,precipitation,rain,payment_type_id,company_id
17403,f1e26e5e2db0d303c0c7da5a13257cec599d76a1,b7ac65c02ae59d09e31fdebdc18a4780593de3157d9c9c...,2023-09-08 09:45:00,2023-09-08 09:45:00,205,0.54,8,32,4.75,0.00,0,0.0,5.25,Credit Card,Chicago Independents,41.890922026,-87.618868355,41.884987192,-87.620992913,17.6,14.8,0.2,0.2,1,13
12190,092e49e8f0b42bb1c80e091cfd9cff7777daf1f8,dc99a6b7916bb2dc16f1a099e29a3caf7dd52ddc34a877...,2023-09-08 10:30:00,2023-09-08 10:30:00,331,0.66,8,8,5.50,0.00,0,1.5,7.00,Cash,Taxicab Insurance Agency Llc,41.899602111,-87.633308037,41.899602111,-87.633308037,17.4,14.5,0.1,0.1,2,11
12316,a597c03e54a826e4caa1d9782c9d85435e91db9c,3b58b6e6c05297a7feacac9a19e049b714eacfa1d49d48...,2023-09-08 00:00:00,2023-09-08 00:15:00,507,1.95,28,32,45.00,0.00,0,0.0,45.00,Cash,Taxicab Insurance Agency Llc,41.874005383,-87.66351755,41.878865584,-87.625192142,18.8,14.2,0.0,0.0,2,11
220,78e8dacd209699151b29969e30607b5ffcb3956e,de8e8659ceb9eb0da842a46b60c0a5207098ac69fc23ab...,2023-09-08 17:45:00,2023-09-08 18:30:00,2908,12.50,76,4,36.50,10.25,0,4.0,51.25,Credit Card,Sun Taxi,41.980264315,-87.913624596,41.975170943,-87.687515515,17.8,20.4,3.1,3.1,1,1
13623,2db9dea8873ef6079abdc744472eae93bd2c699a,94cc07a9d2534052a473c0cb2c5cb2be3ffd131ba7db42...,2023-09-08 07:00:00,2023-09-08 07:45:00,2116,18.49,32,76,45.50,12.20,0,15.0,73.20,Credit Card,City Service,41.880994471,-87.632746489,41.97907082,-87.903039661,17.9,13.0,1.0,1.0,1,8


In [41]:
taxi_trips_with_weather_id.drop(['payment_type', 'company'], axis=1, inplace=True)

In [42]:
taxi_trips_with_weather_id.memory_usage(deep=True).sum() / 1024

13272.544921875

In [43]:
payment_type_master.to_csv(data_out+'payment_type_master.csv', index=False)
company_type_master.to_csv(data_out+'company_master.csv', index=False)


#### Checking the modelling

#### Payment

In [44]:
#Ha új payment type van azt frissíteni kell
payment_type_master = taxi_trips_with_weather['payment_type'].drop_duplicates().reset_index(drop=True)
payment_type_master = pd.DataFrame({
    'payment_type_id': range(1, len(payment_type_master) + 1),
    'payment_type':  payment_type_master
})

In [56]:
payment_type_master

,payment_type_id,payment_type
0,1,Credit Card
1,2,Cash
2,3,Prcard
3,4,Mobile
4,5,Unknown
5,6,No Charge
6,7,Dispute
7,9,X
8,10,Y


In [45]:
#Kézzel megcsináljuk a meglévő típusokra a df-et
#Most azt szimuláljuk, hogy van 2 új típus
new_payment_type_data = [
    {'payment_type': 'Credit Card'},
    {'payment_type': 'X'},
    {'payment_type': 'Y'},
]

#Mapping

new_payment_type_mapping = pd.DataFrame(new_payment_type_data)

In [46]:
new_payment_type_mapping

,payment_type
0,Credit Card
1,X
2,Y


In [47]:
#Megnézzük, hogy a payment type masterben melyik a max index.
payment_type_max_id = payment_type_master['payment_type_id'].max()

In [48]:
#Hogyan lehet bővíteni a meglévő payment type-okat újakkal?
#Készítünk az indexnek egy sorozatot, ami a meglévő max értéktől indúl
#a new payment type hossza + payment type max id-ig
range(payment_type_max_id + 1, payment_type_max_id + len(new_payment_type_mapping)+1)

range(8, 11)

In [49]:
#A mapping táblában új oszlop az id-nak
new_payment_type_mapping['payment_type_id'] = range(payment_type_max_id + 1, payment_type_max_id + len(new_payment_type_mapping)+1)

In [50]:
new_payment_type_mapping

,payment_type,payment_type_id
0,Credit Card,8
1,X,9
2,Y,10


In [51]:
#Ellenőrtés, hogy van-e az eredetiban
new_payment_type_mapping['payment_type'].isin(payment_type_master['payment_type'])

0     True
1    False
2    False
Name: payment_type, dtype: bool

In [52]:
#Szűrjük le a mappinget a False-okra
new_payment_type_mapping[~new_payment_type_mapping['payment_type'].isin(payment_type_master['payment_type'])]


,payment_type,payment_type_id
1,X,9
2,Y,10


In [53]:
new_payment_types = new_payment_type_mapping[~new_payment_type_mapping['payment_type'].isin(payment_type_master['payment_type'])]


In [54]:
if not new_payment_types.empty:
    payment_type_master = pd.concat([payment_type_master, new_payment_types],ignore_index=True )

In [55]:
payment_type_master

,payment_type_id,payment_type
0,1,Credit Card
1,2,Cash
2,3,Prcard
3,4,Mobile
4,5,Unknown
5,6,No Charge
6,7,Dispute
7,9,X
8,10,Y


#### Company data

In [74]:
company_master = taxi_trips_with_weather['company'].drop_duplicates().reset_index(drop=True)
company_master = pd.DataFrame({
    'company_id': range(1, len(company_master) + 1),
    'company':  company_master
})

In [58]:
company_master.tail()

,company_id,company
24,25,2733 - 74600 Benny Jona
25,26,6574 - Babylon Express Inc.
26,27,3556 - 36214 RC Andrews Cab
27,28,5167 - 71969 5167 Taxi Inc
28,29,Petani Cab Corp


In [75]:
company_master.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   company_id  29 non-null     int64 
 1   company     29 non-null     object
dtypes: int64(1), object(1)
memory usage: 596.0+ bytes


In [76]:
new_company_data = [
    {'company': 'Petani Cab Corp'},
    {'company': 'X'},
    {'company': 'Y'},
]

#Mapping

new_company_mapping = pd.DataFrame(new_company_data)

In [77]:
new_company_mapping

,company
0,Petani Cab Corp
1,X
2,Y


In [78]:
company_max_id = company_master['company_id'].max()
new_company_mapping['company_id'] = range(company_max_id + 1, company_max_id + len(new_company_mapping)+1)

In [80]:
new_company_mapping

,company,company_id
0,Petani Cab Corp,30
1,X,31
2,Y,32


In [81]:
#Ellenőrtés, hogy van-e az eredetiban
new_company_mapping['company'].isin(company_master['company'])

0     True
1    False
2    False
Name: company, dtype: bool

In [82]:
#Szűrjük le a mappinget a False-okra
new_company_mapping[~new_company_mapping['company'].isin(company_master['company'])]

,company,company_id
1,X,31
2,Y,32


In [83]:
new_company = new_company_mapping[~new_company_mapping['company'].isin(company_master['company'])]

if not new_company.empty:
    company_master = pd.concat([company_master,new_company],ignore_index=True )

In [84]:
company_master.tail()

,company_id,company
26,27,3556 - 36214 RC Andrews Cab
27,28,5167 - 71969 5167 Taxi Inc
28,29,Petani Cab Corp
29,31,X
30,32,Y
